# Setup

Setting up environment by importing necessary libraries and initializing Spark.

In [1]:
from local_install import setup_environment
setup_environment()

In [2]:
import folium
import pyspark
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
import os
%pylab inline
%pylab notebook

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [3]:
sc = pyspark.SparkContext()
sql = pyspark.sql.SQLContext(sc)

Load data into dataframes.

In [4]:
root_path = "./data"

# file_struct objects
com_file = {'header': "cm_header_file.csv", 'data': "cm.txt" }
can_file = {'header': "cn_header_file.csv", 'data': "cn.txt" }
com_can_link_file = {'header': "ccl_header_file.csv", 'data': "ccl.txt" }
indv_file = {'header': "indiv_header_file.csv", 'data': "itcont.txt" }
pas_file = {'header': "pas2_header_file.csv", 'data': "itpas2.txt" }
com_links_file = {'header': "oth_header_file.csv", 'data': "itoth.txt" }

def load_header(filename):
    """
    Given a header .csv file, return a list containing the names of all columns in the table.
    
    Input:
    filename: a string specifying the header .csv file to load
    
    Output:
    A list containing column names of the table
    """
    with open(filename, "r") as f:
        return [r.replace('\r\n','') for r in f.readline().split(",")]
    
def load_dataframe(file_struct):
    """
    Given a dictionary representing the locations of FEC raw files corresponding to a table,
    load the tables into a Spark DataFrame.
    
    Input:
    file_struct: a dictionary containing the keys 'header' and 'data', where
     'header' contains the name of the `.csv` file specifying the table header file, and
     'data' contains the name of the `.txt` file specifying the table data file
     
    Output:
    A DataFrame which contains the loaded data.
    """
    df = None
    
    lines = sc.textFile(root_path + "/" + file_struct['data'])
    data = lines.map(lambda x: tuple(cell.strip() for cell in x.split("|")))
    
    fields = [StructField(field_name, StringType(), True) for field_name in load_header(root_path + "/" + file_struct['header'])]
    schema = StructType(fields)
    
    df = sql.createDataFrame(data, schema)
    return df.cache()

Initialize dataframes and register temporary SQL tables.

In [5]:
com = load_dataframe(com_file)
can = load_dataframe(can_file)
links = load_dataframe(com_can_link_file)
indv = load_dataframe(indv_file)
pas = load_dataframe(pas_file)
com_links = load_dataframe(com_links_file)

In [6]:
com.registerTempTable("com")
can.registerTempTable("can")
links.registerTempTable("links")
indv.registerTempTable("indv")
pas.registerTempTable("pas")
com_links.registerTempTable("com_links")

# Data Exploration

Let's query for information corresponding to the strings "Clinton", "Sanders", "Trump", and "Cruz".

In [7]:
front_runners = sql.sql("""
SELECT CAND_ID, CAND_NAME, CAND_STATUS, CAND_OFFICE
FROM can
WHERE CAND_NAME LIKE "%CLINTON%"
OR CAND_NAME LIKE "%SANDERS%"
OR CAND_NAME LIKE "%TRUMP%"
OR CAND_NAME LIKE "%CRUZ%"
""")
front_runners.registerTempTable("fr")
front_runners.show(truncate=False)

+---------+-----------------------------------+-----------+-----------+
|CAND_ID  |CAND_NAME                          |CAND_STATUS|CAND_OFFICE|
+---------+-----------------------------------+-----------+-----------+
|H2CA24153|THAYNE, DAVID CRUZ                 |P          |H          |
|H2MI08105|HETRICK, BRIAN CLINTON             |N          |H          |
|H2TX33123|SANDERS, KENNETH                   |P          |H          |
|H6HI01287|DELACRUZ SANTIAGO OSTROV, SHIRLENE |N          |H          |
|H6NY05051|SANDERS, JAMES JR                  |C          |H          |
|H6NY09137|SANDERS, JAIME                     |N          |H          |
|H8OH02041|SANDERS, CHARLES W                 |P          |H          |
|P00003392|CLINTON, HILLARY RODHAM            |C          |P          |
|P60006111|CRUZ, RAFAEL EDWARD "TED"          |C          |P          |
|P60007168|SANDERS, BERNARD                   |C          |P          |
|P60012184|BALLSACK, DONALD TRUMP'S HAIRY KING|N          |P    

Our query results in more candidates than the ones we are looking for. To extract the front-runners we filter using the candidate's office and status.

In [9]:
front_runners = sql.sql("""
SELECT CAND_ID, CAND_NAME, CAND_PCC
FROM can
WHERE CAND_STATUS="C" AND CAND_OFFICE="P" AND 
( CAND_NAME LIKE "%CLINTON%"
OR CAND_NAME LIKE "%SANDERS%"
OR CAND_NAME LIKE "%TRUMP%"
OR CAND_NAME LIKE "%CRUZ%") 
""")
front_runners.registerTempTable("front_runners")
q1 = front_runners.select("CAND_ID", "CAND_NAME", "CAND_PCC")
q1.show(truncate=False)

+---------+-------------------------+---------+
|CAND_ID  |CAND_NAME                |CAND_PCC |
+---------+-------------------------+---------+
|P00003392|CLINTON, HILLARY RODHAM  |C00575795|
|P60006111|CRUZ, RAFAEL EDWARD "TED"|C00574624|
|P60007168|SANDERS, BERNARD         |C00577130|
|P80001571|TRUMP, DONALD J.         |C00580100|
+---------+-------------------------+---------+



The following query gives us the number of contributions by individuals that each front-runner's principal campaign committee has received.

In [11]:
num_indv_contributions = sql.sql("""
SELECT CAND_ID, CAND_NAME, COUNT(*) AS count
FROM indv JOIN front_runners ON CAND_PCC=CMTE_ID
GROUP BY CAND_ID,CAND_NAME
""")

q2 = num_indv_contributions.select("CAND_ID", "CAND_NAME", "count")
q2.show(truncate=False)

+---------+-------------------------+------+
|CAND_ID  |CAND_NAME                |count |
+---------+-------------------------+------+
|P60006111|CRUZ, RAFAEL EDWARD "TED"|427898|
|P80001571|TRUMP, DONALD J.         |40204 |
|P60007168|SANDERS, BERNARD         |882781|
|P00003392|CLINTON, HILLARY RODHAM  |406413|
+---------+-------------------------+------+



... and the total amount corresponding to these contributions. 

In [12]:
indv_contributions_amt = sql.sql("""
SELECT CAND_ID, CAND_NAME, SUM(TRANSACTION_AMT) AS SUM
FROM indv JOIN front_runners ON CAND_PCC=CMTE_ID
GROUP BY CAND_ID,CAND_NAME
""")

q3 = indv_contributions_amt.select("CAND_ID", "CAND_NAME", "SUM")
q3.show(truncate=False)

+---------+-------------------------+-----------+
|CAND_ID  |CAND_NAME                |SUM        |
+---------+-------------------------+-----------+
|P60006111|CRUZ, RAFAEL EDWARD "TED"|5.6260943E7|
|P80001571|TRUMP, DONALD J.         |1.4911975E7|
|P60007168|SANDERS, BERNARD         |6.0064248E7|
|P00003392|CLINTON, HILLARY RODHAM  |1.5295141E8|
+---------+-------------------------+-----------+



This query shows the committees linked to each of the front-runners. 

In [13]:
linked_committees = sql.sql("""
SELECT fr.CAND_ID, CAND_NAME, l.CMTE_ID, CMTE_NM
FROM front_runners fr
JOIN links l
ON fr.CAND_ID = l.CAND_ID
JOIN com c
ON l.CMTE_ID = c.CMTE_ID
""")

q4 = linked_committees.select("CAND_NAME", "CAND_ID", "CMTE_ID", "CMTE_NM")
q4.show(truncate=False)

+-------------------------+---------+---------+-------------------------------------------------------------------------------+
|CAND_NAME                |CAND_ID  |CMTE_ID  |CMTE_NM                                                                        |
+-------------------------+---------+---------+-------------------------------------------------------------------------------+
|CLINTON, HILLARY RODHAM  |P00003392|C00577395|PEOPLE IN COMMAND/PIC                                                          |
|TRUMP, DONALD J.         |P80001571|C00621672|VERMONT TRUMP DELEGATE FUND                                                    |
|SANDERS, BERNARD         |P60007168|C00590646|NEW YORK CAPITAL REGION FOR BRINGING ECONOMIC REVOLUTION NOW INSPIRING EVERYONE|
|SANDERS, BERNARD         |P60007168|C00612549|LORAIN COUNTY FORWARD                                                          |
|CLINTON, HILLARY RODHAM  |P00003392|C00575795|HILLARY FOR AMERICA                                      

... the number of contributions by committees that each front-runner has received

In [15]:
num_com_contributions = sql.sql("""
SELECT CAND_NAME, COUNT(*) AS count
FROM pas JOIN front_runners ON pas.CAND_ID = front_runners.CAND_ID
GROUP BY CAND_NAME
""")

q5 = num_com_contributions.select("CAND_NAME", "count")
q5.show(truncate=False)

+-------------------------+-----+
|CAND_NAME                |count|
+-------------------------+-----+
|CLINTON, HILLARY RODHAM  |6567 |
|TRUMP, DONALD J.         |1921 |
|SANDERS, BERNARD         |2250 |
|CRUZ, RAFAEL EDWARD "TED"|1581 |
+-------------------------+-----+



... and the total amount corresponding to these contributions

In [16]:
com_contributions_amt = sql.sql("""
SELECT CAND_NAME, SUM(TRANSACTION_AMT) AS SUM
FROM pas JOIN front_runners ON pas.CAND_ID = front_runners.CAND_ID
GROUP BY CAND_NAME
""")

q6 = com_contributions_amt.select("CAND_NAME", "SUM")
q6.show(truncate=False)

+-------------------------+-----------+
|CAND_NAME                |SUM        |
+-------------------------+-----------+
|CLINTON, HILLARY RODHAM  |2.4628653E7|
|TRUMP, DONALD J.         |4.4360356E7|
|SANDERS, BERNARD         |4410791.0  |
|CRUZ, RAFAEL EDWARD "TED"|2.880968E7 |
+-------------------------+-----------+

